In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, Conv2D,MaxPool2D,ZeroPadding2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import vgg16
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns

#for model deployement 
import os
import cv2
import numpy as np
from tensorflow.keras.models import model_from_json
from tensorflow.keras.utils import img_to_array

In [3]:
model = model_from_json(open("/Users/gautamprakash/Downloads/model.json", "r").read())
model.load_weights('/Users/gautamprakash/Downloads/best_model.h5')
face_haar_cascade = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')

2023-06-20 23:52:20.682353: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
[ERROR:0@3.860] global /Users/runner/work/opencv-python/opencv-python/opencv/modules/core/src/persistence.cpp (505) open Can't open file: './haarcascade_frontalface_default.xml' in read mode


In [5]:
import cv2
import numpy as np
from keras.models import model_from_json
from tensorflow.keras.utils import img_to_array

# Load the model from JSON file
model_json = open("model.json", "r").read()
model = model_from_json(model_json)
model.load_weights('best_model.h5')

# Load the face cascade classifier
face_haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Start the webcam
cap = cv2.VideoCapture(0)

while True:
    # Read the video frame
    ret, frame = cap.read()
    
    if not ret:
        break

    # Convert the frame to grayscale
    gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale frame
    faces_detected = face_haar_cascade.detectMultiScale(gray_image)

    for (x, y, w, h) in faces_detected:
        # Draw a rectangle around the detected face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        # Crop and resize the face region
        roi_gray = gray_image[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (48, 48))

        # Convert the image to array and expand dimensions
        image_pixels = img_to_array(roi_gray)
        image_pixels = np.expand_dims(image_pixels, axis=0)

        # Perform model prediction
        predictions = model.predict(image_pixels)
        max_index = np.argmax(predictions[0])
        
        emotion_detection = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
        emotion_prediction = emotion_detection[max_index]

        # Draw the predicted emotion label on the frame
        cv2.putText(frame, emotion_prediction, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Emotion Detector', frame)

    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close the windows
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 60ms/step


1/1 [==============================] - 0s 28ms/step


In [5]:
from flask import Flask, render_template, Response
import cv2
app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, Conv2D,MaxPool2D,ZeroPadding2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import vgg16
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns

#for model deployement 
import os
import cv2
import numpy as np
from tensorflow.keras.models import model_from_json
from tensorflow.keras.utils import img_to_array
from keras.models import model_from_json 
model = model_from_json(open("model.json", "r").read())
model.load_weights('best_model.h5')
face_haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def generate_frames():
    
        # Start the webcam
    cap = cv2.VideoCapture(0)

    while True:
        # Read the video frame
        ret, frame = cap.read()

        if not ret:
            break

        # Convert the frame to grayscale
        gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the grayscale frame
        faces_detected = face_haar_cascade.detectMultiScale(gray_image)

        for (x, y, w, h) in faces_detected:
            # Draw a rectangle around the detected face
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

            # Crop and resize the face region
            roi_gray = gray_image[y:y+h, x:x+w]
            roi_gray = cv2.resize(roi_gray, (48, 48))

            # Convert the image to array and expand dimensions
            image_pixels = img_to_array(roi_gray)
            image_pixels = np.expand_dims(image_pixels, axis=0)

            # Perform model prediction
            predictions = model.predict(image_pixels)
            max_index = np.argmax(predictions[0])

            emotion_detection = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
            emotion_prediction = emotion_detection[max_index]

            # Draw the predicted emotion label on the frame
            cv2.putText(frame, emotion_prediction, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Display the frame
        cv2.imshow('Emotion Detector', frame)

        # Exit if 'b' is pressed
        if cv2.waitKey(1) & 0xFF == ord('b'):
            break

    # Release the video capture and close the windows
    cap.release()
    cv2.destroyAllWindows()
@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(debug=True, port=5001)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "/opt/anaconda3/envs/env_tensorflow/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/envs/env_tensorflow/lib/python3.9/site-packages/traitlets/config/application.py", line 845, in launch_instance
    app.initialize(argv)
  File "/opt/anaconda3/envs/env_tensorflow/lib/python3.9/site-packages/traitlets/config/application.py", line 88, in inner
    return method(app, *args, **kwargs)
  File "/opt/anaconda3/envs/env_tensorflow/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 666, in initialize
    self.init_sockets()
  File "/opt/anaconda3/envs/env_tensorflow/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 307, in init_sockets
    self.shell_port = self._bind_socket(self.shell_socket, self.shell_port)
  File "/opt/anaconda3/envs/env_tensorflow/lib/python3.9/site-packages/ipy

SystemExit: 1

In [3]:
pip install flask

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 715.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 2.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 2.0.3
    Uninstalling Werkzeug-2.0.3:
      Successfully uninstalled Werkzeug-2.0.3
  Attempting uninstall: click
    Found existing installation: click 8.0.4
    Uninstalling click-8.0.4:
      Successfully uninstalled click-8.0.4
  Attempting uninstall: blinker
    Found existing installation: blinker 1.4
    Uninstalling blinker-1.4:
      Successfully uninstalled blinker-1.4
Note: you may need to restart the kernel to use updated packages.


In [6]:
%tb

SystemExit: 1

# Working code:

In [ ]:
from flask import Flask, render_template, Response
import cv2
import numpy as np
from tensorflow.keras.models import model_from_json
from tensorflow.keras.utils import img_to_array

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/carousel')
def carousel():
    return render_template('carousel.html')

# Load the model
json_file = open('better_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights('better_model.h5')

face_haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def generate_frames():
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()

        if not ret:
            break

        gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces_detected = face_haar_cascade.detectMultiScale(gray_image)

        for (x, y, w, h) in faces_detected:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

            roi_gray = gray_image[y:y+h, x:x+w]
            roi_gray = cv2.resize(roi_gray, (48, 48))

            image_pixels = img_to_array(roi_gray)
            image_pixels = np.expand_dims(image_pixels, axis=0)

            predictions = model.predict(image_pixels)
            max_index = np.argmax(predictions[0])

            emotion_detection = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
            emotion_prediction = emotion_detection[max_index]

            cv2.putText(frame, emotion_prediction, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        ret, buffer = cv2.imencode('.jpg', frame)
        frame_bytes = buffer.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame_bytes + b'\r\n')

    cap.release()
    cv2.destroyAllWindows()

@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(port=5000)


2023-06-22 19:45:26.655608: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [22/Jun/2023 19:45:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2023 19:45:49] "GET /carousel HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2023 19:45:49] "GET /static/img-3.jpg HTTP/1.1" 304 -
127.0.0.1 - - [22/Jun/2023 19:45:49] "GET /static/cat-2.jpg HTTP/1.1" 304 -
127.0.0.1 - - [22/Jun/2023 19:45:49] "GET /static/cat-1.jpg HTTP/1.1" 304 -
127.0.0.1 - - [22/Jun/2023 19:45:54] "GET /video_feed HTTP/1.1" 200 -


1/1 [==============================] - 0s 27ms/step


1/1 [==============================] - 0s 27ms/step


In [8]:
%tb

SystemExit: 1

# Improved code:

In [ ]:
from flask import Flask, render_template, Response, redirect, url_for
import cv2
import numpy as np
from tensorflow.keras.models import model_from_json
from tensorflow.keras.utils import img_to_array

app = Flask(__name__)

# Initialize global variables
emotion_predictions = []
model_running = False

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/carousel')
def carousel():
    global model_running
    model_running = True
    return render_template('carousel.html')


@app.route('/stop_model', methods=['GET'])
def stop_model():
    try:
        emotion_predictions = []
        global model_running
        model_running = False
        return redirect(url_for('predictions'))
    except Exception as e:
        return str(e)

@app.route('/predictions')
def predictions():
    global emotion_predictions
    prediction_data = emotion_predictions
    emotion_predictions = []
    return render_template('predictions.html', data=prediction_data)



# Load the model
json_file = open('better_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights('better_model.h5')

face_haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def generate_frames():
    global model_running
    global emotion_predictions 
    cap = cv2.VideoCapture(0)
    while model_running:
        ret, frame = cap.read()

        if not ret:
            break

        gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces_detected = face_haar_cascade.detectMultiScale(gray_image)

        for (x, y, w, h) in faces_detected:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

            roi_gray = gray_image[y:y+h, x:x+w]
            roi_gray = cv2.resize(roi_gray, (48, 48))

            image_pixels = img_to_array(roi_gray)
            image_pixels = np.expand_dims(image_pixels, axis=0)

            model_output = model.predict(image_pixels)
            max_index = np.argmax(model_output[0])

            emotion_detection = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
            emotion_prediction = emotion_detection[max_index]

            cv2.putText(frame, emotion_prediction, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            print(f"emotion_prediction = {emotion_prediction}")
            if model_running:
                emotion_predictions.append(emotion_prediction)

        ret, buffer = cv2.imencode('.jpg', frame)
        frame_bytes = buffer.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame_bytes + b'\r\n')
    cap.release()
    cv2.destroyAllWindows()

@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(port=5000)


2023-06-23 02:06:14.174114: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [23/Jun/2023 02:06:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2023 02:06:20] "GET /carousel HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2023 02:06:20] "GET /static/cat-1.jpg HTTP/1.1" 304 -
127.0.0.1 - - [23/Jun/2023 02:06:20] "GET /static/cat-2.jpg HTTP/1.1" 304 -
127.0.0.1 - - [23/Jun/2023 02:06:20] "GET /static/cat-2.jpg HTTP/1.1" 304 -
127.0.0.1 - - [23/Jun/2023 02:06:20] "GET /static/cat-1.jpg HTTP/1.1" 304 -
127.0.0.1 - - [23/Jun/2023 02:06:20] "GET /static/img-3.jpg HTTP/1.1" 304 -
127.0.0.1 - - [23/Jun/2023 02:06:20] "GET /static/img-3.jpg HTTP/1.1" 304 -
127.0.0.1 - - [23/Jun/2023 02:06:24] "GET /video_feed HTTP/1.1" 200 -


1/1 [==============================] - 0s 128ms/step
emotion_prediction = sad
1/1 [==============================] - 0s 27ms/step
emotion_prediction = sad
1/1 [==============================] - 0s 27ms/step
emotion_prediction = sad
1/1 [==============================] - 0s 27ms/step
emotion_prediction = angry
1/1 [==============================] - 0s 27ms/step
emotion_prediction = happy
1/1 [==============================] - 0s 27ms/step
emotion_prediction = happy
1/1 [==============================] - 0s 28ms/step
emotion_prediction = angry
1/1 [==============================] - 0s 27ms/step
emotion_prediction = fear
1/1 [==============================] - 0s 28ms/step
emotion_prediction = happy
1/1 [==============================] - 0s 27ms/step
emotion_prediction = sad
1/1 [==============================] - 0s 27ms/step
emotion_prediction = sad
1/1 [==============================] - 0s 27ms/step
emotion_prediction = sad
1/1 [==============================] - 0s 27ms/step
emotion_pred

1/1 [==============================] - 0s 27ms/step
emotion_prediction = angry
1/1 [==============================] - 0s 27ms/step
emotion_prediction = happy
1/1 [==============================] - 0s 28ms/step
emotion_prediction = sad
1/1 [==============================] - 0s 27ms/step
emotion_prediction = neutral
1/1 [==============================] - 0s 27ms/step
emotion_prediction = happy
1/1 [==============================] - 0s 28ms/step
emotion_prediction = neutral
1/1 [==============================] - 0s 27ms/step
emotion_prediction = happy
1/1 [==============================] - 0s 27ms/step
emotion_prediction = neutral
1/1 [==============================] - 0s 27ms/step
emotion_prediction = happy
1/1 [==============================] - 0s 28ms/step
emotion_prediction = happy
1/1 [==============================] - 0s 27ms/step
emotion_prediction = neutral
1/1 [==============================] - 0s 28ms/step
emotion_prediction = angry
1/1 [==============================] - 0s 27ms

127.0.0.1 - - [23/Jun/2023 02:06:41] "GET /stop_model HTTP/1.1" 302 -
127.0.0.1 - - [23/Jun/2023 02:06:41] "GET /predictions HTTP/1.1" 200 -


emotion_prediction = happy
1/1 [==============================] - 0s 38ms/step
emotion_prediction = neutral
1/1 [==============================] - 0s 29ms/step
emotion_prediction = happy
1/1 [==============================] - 0s 28ms/step
emotion_prediction = happy
1/1 [==============================] - 0s 27ms/step
emotion_prediction = happy


# Trying with percentage:

In [ ]:
from flask import Flask, render_template, Response, redirect, url_for
import cv2
import numpy as np
from tensorflow.keras.models import model_from_json
from tensorflow.keras.utils import img_to_array
from collections import Counter
from datetime import datetime

app = Flask(__name__)

# Initialize global variables
emotion_predictions = []
model_running = False

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/carousel')
def carousel():
    global model_running
    model_running = True
    return render_template('carousel.html')


@app.route('/stop_model', methods=['GET'])
def stop_model():
    try:
        emotion_predictions = []
        global model_running
        model_running = False
        return redirect(url_for('predictions'))
    except Exception as e:
        return str(e)

@app.route('/predictions')
def predictions():
    global emotion_predictions
    counts = Counter(emotion_predictions)
    total = len(emotion_predictions)
    percentages = {emotion: round(count / total * 100, 2) for emotion, count in counts.items()}
    emotion_predictions = []
    return render_template('predictions.html', data=percentages)



# Load the model
json_file = open('better_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights('better_model.h5')

face_haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def generate_frames():
    global model_running
    global emotion_predictions 
    cap = cv2.VideoCapture(0)
    while model_running:
        ret, frame = cap.read()

        if not ret:
            break

        gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces_detected = face_haar_cascade.detectMultiScale(gray_image)

        for (x, y, w, h) in faces_detected:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

            roi_gray = gray_image[y:y+h, x:x+w]
            roi_gray = cv2.resize(roi_gray, (48, 48))

            image_pixels = img_to_array(roi_gray)
            image_pixels = np.expand_dims(image_pixels, axis=0)

            model_output = model.predict(image_pixels)
            max_index = np.argmax(model_output[0])

            emotion_detection = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
            emotion_prediction = emotion_detection[max_index]

            cv2.putText(frame, emotion_prediction, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            print(f"emotion_prediction = {emotion_prediction}")
            if model_running:
                emotion_predictions.append(emotion_prediction)

        ret, buffer = cv2.imencode('.jpg', frame)
        frame_bytes = buffer.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame_bytes + b'\r\n')
    cap.release()
    cv2.destroyAllWindows()

@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(port=5000)


2023-06-23 19:00:06.757120: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [23/Jun/2023 19:00:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2023 19:00:14] "GET /carousel HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2023 19:00:14] "GET /static/img-3.jpg HTTP/1.1" 304 -
127.0.0.1 - - [23/Jun/2023 19:00:14] "GET /static/cat-2.jpg HTTP/1.1" 304 -
127.0.0.1 - - [23/Jun/2023 19:00:14] "GET /static/cat-1.jpg HTTP/1.1" 304 -
127.0.0.1 - - [23/Jun/2023 19:00:19] "GET /video_feed HTTP/1.1" 200 -


1/1 [==============================] - 0s 131ms/step
emotion_prediction = neutral
1/1 [==============================] - 0s 28ms/step
emotion_prediction = neutral
1/1 [==============================] - 0s 28ms/step
emotion_prediction = neutral
1/1 [==============================] - 0s 28ms/step
emotion_prediction = neutral
1/1 [==============================] - 0s 27ms/step
emotion_prediction = fear
1/1 [==============================] - 0s 30ms/step
emotion_prediction = neutral
1/1 [==============================] - 0s 27ms/step
emotion_prediction = neutral
1/1 [==============================] - 0s 27ms/step
emotion_prediction = neutral
1/1 [==============================] - 0s 29ms/step
emotion_prediction = fear
1/1 [==============================] - 0s 28ms/step
emotion_prediction = fear
1/1 [==============================] - 0s 27ms/step
emotion_prediction = fear
1/1 [==============================] - 0s 31ms/step
emotion_prediction = neutral
1/1 [==============================] - 

1/1 [==============================] - 0s 28ms/step
emotion_prediction = fear
1/1 [==============================] - 0s 28ms/step
emotion_prediction = neutral
1/1 [==============================] - 0s 27ms/step
emotion_prediction = neutral
1/1 [==============================] - 0s 28ms/step
emotion_prediction = neutral
1/1 [==============================] - 0s 32ms/step
emotion_prediction = neutral
1/1 [==============================] - 0s 28ms/step
emotion_prediction = neutral
1/1 [==============================] - 0s 27ms/step
emotion_prediction = fear
1/1 [==============================] - 0s 27ms/step
emotion_prediction = fear
1/1 [==============================] - 0s 28ms/step
emotion_prediction = angry
1/1 [==============================] - 0s 28ms/step
emotion_prediction = fear
1/1 [==============================] - 0s 31ms/step
emotion_prediction = fear


127.0.0.1 - - [23/Jun/2023 19:00:33] "GET /stop_model HTTP/1.1" 302 -
127.0.0.1 - - [23/Jun/2023 19:00:33] "GET /predictions HTTP/1.1" 200 -


1/1 [==============================] - 0s 38ms/step
emotion_prediction = neutral


# Adding more functionalities:

In [ ]:
from flask import Flask, render_template, Response, redirect, url_for
import cv2
import numpy as np
from tensorflow.keras.models import model_from_json
from tensorflow.keras.utils import img_to_array
from collections import Counter
from datetime import datetime
from jinja2 import Environment, FileSystemLoader, select_autoescape
from collections import defaultdict

app = Flask(__name__)

# Initialize global variables
emotion_predictions = []
model_running = False

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/carousel')
def carousel():
    global model_running
    model_running = True
    return render_template('carousel.html')


@app.route('/stop_model', methods=['GET'])
def stop_model():
    try:
        emotion_predictions = []
        global model_running
        model_running = False
        return redirect(url_for('predictions'))
    except Exception as e:
        return str(e)

@app.route('/predictions')
def predictions():
    global emotion_predictions
    data = []
    max_percentages = []
    default_dict = {emotion: 0.0 for emotion in ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']}
    for image_predictions in emotion_predictions:
        total = sum(image_predictions.values())
        print(f"total: {total}")  # debug print
        print(f"image_predictions: {image_predictions}")  # debug print
        percentages = {emotion: round(count / total * 100, 2) for emotion, count in image_predictions.items()}
        updated_dict = default_dict.copy()
        updated_dict.update(percentages)
        print(f"updated_dict: {updated_dict}")  # debug print
        max_percentages.append(max(updated_dict.values()))  
        data.append(updated_dict)
    emotion_predictions = []
    print(f"data: {data}")  # debug print
    return render_template('predictions.html', percentage_predictions=data, max_percentages=max_percentages)

def max_emotion(image_predictions):
    return max(image_predictions, key=image_predictions.get)

# Set up Jinja environment and custom filters
env = Environment(
    loader=FileSystemLoader('templates'),  # change this line
    autoescape=select_autoescape(['html', 'xml']),
    trim_blocks=True,
    lstrip_blocks=True,
)
env.filters['max_emotion'] = max_emotion

# Load the model
json_file = open('better_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights('better_model.h5')

face_haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def generate_frames():
    global model_running
    global emotion_predictions 
    cap = cv2.VideoCapture(0)
    image_number = 1
    start_time = datetime.now()
    while model_running:
        
        ret, frame = cap.read()

        if not ret:
            break

        gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces_detected = face_haar_cascade.detectMultiScale(gray_image)
        for (x, y, w, h) in faces_detected:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

            roi_gray = gray_image[y:y+h, x:x+w]
            roi_gray = cv2.resize(roi_gray, (48, 48))

            image_pixels = img_to_array(roi_gray)
            image_pixels = np.expand_dims(image_pixels, axis=0)

            model_output = model.predict(image_pixels)
            max_index = np.argmax(model_output[0])

            emotion_detection = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
            emotion_prediction = emotion_detection[max_index]

            cv2.putText(frame, emotion_prediction, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            print(f"emotion_prediction = {emotion_prediction}")
            if model_running:
                # Check if 3 seconds have passed
                if (datetime.now() - start_time).seconds >= 3:
                    # Move to next image
                    image_number += 1
                    start_time = datetime.now()

                # Append the prediction to the correct image dictionary
                if len(emotion_predictions) < image_number:
                    emotion_predictions.append({})

                emotion_predictions[image_number - 1][emotion_prediction] = emotion_predictions[image_number - 1].get(emotion_prediction, 0) + 1
                print(f"emotion_predictions: {emotion_predictions}")  # debug print

        ret, buffer = cv2.imencode('.jpg', frame)
        frame_bytes = buffer.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame_bytes + b'\r\n')
    cap.release()
    cv2.destroyAllWindows()

@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(port=5000)


2023-06-24 00:02:41.681283: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [24/Jun/2023 00:02:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2023 00:02:46] "GET /carousel HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2023 00:02:46] "GET /static/img-3.jpg HTTP/1.1" 304 -
127.0.0.1 - - [24/Jun/2023 00:02:46] "GET /static/cat-1.jpg HTTP/1.1" 304 -
127.0.0.1 - - [24/Jun/2023 00:02:46] "GET /static/cat-2.jpg HTTP/1.1" 304 -
127.0.0.1 - - [24/Jun/2023 00:02:51] "GET /video_feed HTTP/1.1" 200 -


1/1 [==============================] - 0s 135ms/step
emotion_prediction = fear
emotion_predictions: [{'fear': 1}]
1/1 [==============================] - 0s 28ms/step
emotion_prediction = fear
emotion_predictions: [{'fear': 2}]
1/1 [==============================] - 0s 29ms/step
emotion_prediction = neutral
emotion_predictions: [{'fear': 2, 'neutral': 1}]
1/1 [==============================] - 0s 29ms/step
emotion_prediction = fear
emotion_predictions: [{'fear': 3, 'neutral': 1}]
1/1 [==============================] - 0s 28ms/step
emotion_prediction = happy
emotion_predictions: [{'fear': 3, 'neutral': 1, 'happy': 1}]
1/1 [==============================] - 0s 27ms/step
emotion_prediction = neutral
emotion_predictions: [{'fear': 3, 'neutral': 2, 'happy': 1}]
1/1 [==============================] - 0s 27ms/step
emotion_prediction = neutral
emotion_predictions: [{'fear': 3, 'neutral': 3, 'happy': 1}]
1/1 [==============================] - 0s 27ms/step
emotion_prediction = sad
emotion_predict

1/1 [==============================] - 0s 28ms/step
emotion_prediction = neutral
emotion_predictions: [{'fear': 5, 'neutral': 14, 'happy': 6, 'sad': 5, 'angry': 3}, {'neutral': 14, 'happy': 2, 'fear': 2}]
1/1 [==============================] - 0s 28ms/step
emotion_prediction = neutral
emotion_predictions: [{'fear': 5, 'neutral': 14, 'happy': 6, 'sad': 5, 'angry': 3}, {'neutral': 15, 'happy': 2, 'fear': 2}]
1/1 [==============================] - 0s 27ms/step
emotion_prediction = happy
emotion_predictions: [{'fear': 5, 'neutral': 14, 'happy': 6, 'sad': 5, 'angry': 3}, {'neutral': 15, 'happy': 3, 'fear': 2}]
1/1 [==============================] - 0s 27ms/step
emotion_prediction = neutral
emotion_predictions: [{'fear': 5, 'neutral': 14, 'happy': 6, 'sad': 5, 'angry': 3}, {'neutral': 16, 'happy': 3, 'fear': 2}]
1/1 [==============================] - 0s 27ms/step
emotion_prediction = neutral
emotion_predictions: [{'fear': 5, 'neutral': 14, 'happy': 6, 'sad': 5, 'angry': 3}, {'neutral': 17, '

1/1 [==============================] - 0s 30ms/step
emotion_prediction = neutral
emotion_predictions: [{'fear': 5, 'neutral': 14, 'happy': 6, 'sad': 5, 'angry': 3}, {'neutral': 20, 'happy': 3, 'fear': 2}, {'neutral': 14, 'fear': 5, 'happy': 3, 'sad': 1, 'angry': 1, 'surprise': 1}, {'neutral': 1}]
1/1 [==============================] - 0s 34ms/step
emotion_prediction = neutral
emotion_predictions: [{'fear': 5, 'neutral': 14, 'happy': 6, 'sad': 5, 'angry': 3}, {'neutral': 20, 'happy': 3, 'fear': 2}, {'neutral': 14, 'fear': 5, 'happy': 3, 'sad': 1, 'angry': 1, 'surprise': 1}, {'neutral': 2}]
1/1 [==============================] - 0s 30ms/step
emotion_prediction = neutral
emotion_predictions: [{'fear': 5, 'neutral': 14, 'happy': 6, 'sad': 5, 'angry': 3}, {'neutral': 20, 'happy': 3, 'fear': 2}, {'neutral': 14, 'fear': 5, 'happy': 3, 'sad': 1, 'angry': 1, 'surprise': 1}, {'neutral': 3}]
1/1 [==============================] - 0s 29ms/step
emotion_prediction = fear
emotion_predictions: [{'fear

1/1 [==============================] - 0s 28ms/step
emotion_prediction = neutral
emotion_predictions: [{'fear': 5, 'neutral': 14, 'happy': 6, 'sad': 5, 'angry': 3}, {'neutral': 20, 'happy': 3, 'fear': 2}, {'neutral': 14, 'fear': 5, 'happy': 3, 'sad': 1, 'angry': 1, 'surprise': 1}, {'neutral': 19, 'fear': 2, 'happy': 1, 'angry': 5}]
1/1 [==============================] - 0s 28ms/step
emotion_prediction = happy
emotion_predictions: [{'fear': 5, 'neutral': 14, 'happy': 6, 'sad': 5, 'angry': 3}, {'neutral': 20, 'happy': 3, 'fear': 2}, {'neutral': 14, 'fear': 5, 'happy': 3, 'sad': 1, 'angry': 1, 'surprise': 1}, {'neutral': 19, 'fear': 2, 'happy': 2, 'angry': 5}]
1/1 [==============================] - 0s 29ms/step
emotion_prediction = neutral
emotion_predictions: [{'fear': 5, 'neutral': 14, 'happy': 6, 'sad': 5, 'angry': 3}, {'neutral': 20, 'happy': 3, 'fear': 2}, {'neutral': 14, 'fear': 5, 'happy': 3, 'sad': 1, 'angry': 1, 'surprise': 1}, {'neutral': 20, 'fear': 2, 'happy': 2, 'angry': 5}]


127.0.0.1 - - [24/Jun/2023 00:03:03] "GET /stop_model HTTP/1.1" 302 -
127.0.0.1 - - [24/Jun/2023 00:03:03] "GET /predictions HTTP/1.1" 200 -


emotion_prediction = fear
emotion_predictions: [{'fear': 5, 'neutral': 14, 'happy': 6, 'sad': 5, 'angry': 3}, {'neutral': 20, 'happy': 3, 'fear': 2}, {'neutral': 14, 'fear': 5, 'happy': 3, 'sad': 1, 'angry': 1, 'surprise': 1}, {'neutral': 20, 'fear': 2, 'happy': 2, 'angry': 5}, {'neutral': 4, 'fear': 1}]
total: 33
image_predictions: {'fear': 5, 'neutral': 14, 'happy': 6, 'sad': 5, 'angry': 3}
updated_dict: {'angry': 9.09, 'disgust': 0.0, 'fear': 15.15, 'happy': 18.18, 'sad': 15.15, 'surprise': 0.0, 'neutral': 42.42}
total: 25
image_predictions: {'neutral': 20, 'happy': 3, 'fear': 2}
updated_dict: {'angry': 0.0, 'disgust': 0.0, 'fear': 8.0, 'happy': 12.0, 'sad': 0.0, 'surprise': 0.0, 'neutral': 80.0}
total: 25
image_predictions: {'neutral': 14, 'fear': 5, 'happy': 3, 'sad': 1, 'angry': 1, 'surprise': 1}
updated_dict: {'angry': 4.0, 'disgust': 0.0, 'fear': 20.0, 'happy': 12.0, 'sad': 4.0, 'surprise': 4.0, 'neutral': 56.0}
total: 29
image_predictions: {'neutral': 20, 'fear': 2, 'happy': 2

# Updated code (Lokesh and Debu):

In [1]:
from flask import Flask, render_template, Response, request, redirect, url_for
import cv2
import numpy as np
from tensorflow.keras.models import model_from_json
from tensorflow.keras.utils import img_to_array
from collections import Counter
from datetime import datetime
from jinja2 import Environment, FileSystemLoader, select_autoescape
from collections import defaultdict

app = Flask(__name__)

# Initialize global variables
emotion_predictions = []
model_running = False
start_predict = False

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/carousel')
def carousel():
    global model_running
    model_running = True
    return render_template('carousel.html')


@app.route('/stop_model', methods=['GET'])
def stop_model():
    try:
        emotion_predictions = []
        global model_running
        model_running = False
        return redirect(url_for('predictions'))
    except Exception as e:
        return str(e)

@app.route('/predictions')
def predictions():
    global emotion_predictions
    data = []
    max_percentages = []
    default_dict = {emotion: 0.0 for emotion in ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']}
    for image_predictions in emotion_predictions:
        total = sum(image_predictions.values())
        print(f"total: {total}")  # debug print
        print(f"image_predictions: {image_predictions}")  # debug print
        percentages = {emotion: round(count / total * 100, 2) for emotion, count in image_predictions.items()}
        updated_dict = default_dict.copy()
        updated_dict.update(percentages)
        print(f"updated_dict: {updated_dict}")  # debug print
        max_percentages.append(max(updated_dict.values()))  
        data.append(updated_dict)
    emotion_predictions = []
    print(f"data: {data}")  # debug print
    return render_template('predictions.html', percentage_predictions=data, max_percentages=max_percentages)

def max_emotion(image_predictions):
    return max(image_predictions, key=image_predictions.get)

# Set up Jinja environment and custom filters
env = Environment(
    loader=FileSystemLoader('templates'),  # change this line
    autoescape=select_autoescape(['html', 'xml']),
    trim_blocks=True,
    lstrip_blocks=True,
)
env.filters['max_emotion'] = max_emotion

# Load the model
json_file = open('better_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights('better_model.h5')

face_haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def generate_frames():
    global model_running
    global emotion_predictions 
    global start_predict
    cap = cv2.VideoCapture(0)
    image_number = 0
    start_time = datetime.now()
    while model_running:
        
        ret, frame = cap.read()

        if not ret:
            break
        
        
        gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces_detected = face_haar_cascade.detectMultiScale(gray_image)
        
        for (x, y, w, h) in faces_detected:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

            roi_gray = gray_image[y:y+h, x:x+w]
            roi_gray = cv2.resize(roi_gray, (48, 48))

            image_pixels = img_to_array(roi_gray)
            image_pixels = np.expand_dims(image_pixels, axis=0)

            if start_predict == True :
                model_output = model.predict(image_pixels)
                max_index = np.argmax(model_output[0])

                emotion_detection = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
                emotion_prediction = emotion_detection[max_index]

                cv2.putText(frame, emotion_prediction, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                print(f"emotion_prediction = {emotion_prediction}")
                if model_running:
                    # Check if 3 seconds have passed
                    if (datetime.now() - start_time).seconds >= 3:
                        # Move to next image
                        image_number += 1
                        start_time = datetime.now()

                    # Append the prediction to the correct image dictionary
                    if len(emotion_predictions) < image_number:
                        emotion_predictions.append({})

                    emotion_predictions[image_number - 1][emotion_prediction] = emotion_predictions[image_number - 1].get(emotion_prediction, 0) + 1
                    print(f"emotion_predictions: {emotion_predictions}")  # debug print
  
        ret, buffer = cv2.imencode('.jpg', frame)
        frame_bytes = buffer.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame_bytes + b'\r\n')
        
        if image_number == 17:
            break
    cap.release()
    cv2.destroyAllWindows()

@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')


@app.route('/button_clicked', methods=['POST'])
def button_clicked():
    global start_predict
    start_predict = True
    return ''


if __name__ == '__main__':
    app.run(port=5000)

2023-06-28 01:41:36.452983: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [28/Jun/2023 01:41:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 01:41:45] "GET /carousel HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 01:41:45] "GET /static/img-4.jpg HTTP/1.1" 304 -
127.0.0.1 - - [28/Jun/2023 01:41:45] "GET /static/cat-2.jpg HTTP/1.1" 304 -
127.0.0.1 - - [28/Jun/2023 01:41:45] "GET /static/img-5.jpg HTTP/1.1" 304 -
127.0.0.1 - - [28/Jun/2023 01:41:45] "GET /static/img-7.jpg HTTP/1.1" 304 -
127.0.0.1 - - [28/Jun/2023 01:41:45] "GET /static/img-3.jpg HTTP/1.1" 304 -
127.0.0.1 - - [28/Jun/2023 01:41:45] "GET /static/img-6.jpg HTTP/1.1" 304 -
127.0.0.1 - - [28/Jun/2023 01:41:45] "GET /static/img-9.jpg HTTP/1.1" 304 -
127.0.0.1 - - [28/Jun/2023 01:41:45] "GET /static/img-10.jpg HTTP/1.1" 304 -
127.0.0.1 - - [28/Jun/2023 01:41:45] "GET /static/img-8.jpg HTTP/1.1" 304 -
127.0.0.1 - - [28/Jun/2023 01:41:45] "GET /static/img-12.jpg HTTP/1.1" 304 -
127.0.0.1 - - [28/Jun/2023 01:41:45] "GET /static/img

1/1 [==============================] - 0s 146ms/step
emotion_prediction = happy
emotion_predictions: [{'happy': 1}]
1/1 [==============================] - 0s 28ms/step
emotion_prediction = fear
emotion_predictions: [{'happy': 1, 'fear': 1}]
1/1 [==============================] - 0s 28ms/step
emotion_prediction = happy
emotion_predictions: [{'happy': 2, 'fear': 1}]
1/1 [==============================] - 0s 28ms/step
emotion_prediction = fear
emotion_predictions: [{'happy': 2, 'fear': 2}]
1/1 [==============================] - 0s 28ms/step
emotion_prediction = happy
emotion_predictions: [{'happy': 3, 'fear': 2}]
1/1 [==============================] - 0s 31ms/step
emotion_prediction = neutral
emotion_predictions: [{'happy': 3, 'fear': 2, 'neutral': 1}]
1/1 [==============================] - 0s 30ms/step
emotion_prediction = happy
emotion_predictions: [{'happy': 4, 'fear': 2, 'neutral': 1}]
1/1 [==============================] - 0s 31ms/step
emotion_prediction = fear
emotion_predictions: [

1/1 [==============================] - 0s 27ms/step
emotion_prediction = fear
emotion_predictions: [{'happy': 14, 'fear': 7, 'neutral': 2, 'sad': 2}, {'happy': 13, 'neutral': 7, 'fear': 6}]
1/1 [==============================] - 0s 29ms/step
emotion_prediction = happy
emotion_predictions: [{'happy': 14, 'fear': 7, 'neutral': 2, 'sad': 2}, {'happy': 14, 'neutral': 7, 'fear': 6}]
1/1 [==============================] - 0s 28ms/step
emotion_prediction = happy
emotion_predictions: [{'happy': 14, 'fear': 7, 'neutral': 2, 'sad': 2}, {'happy': 14, 'neutral': 7, 'fear': 6}, {'happy': 1}]
1/1 [==============================] - 0s 28ms/step
emotion_prediction = happy
emotion_predictions: [{'happy': 14, 'fear': 7, 'neutral': 2, 'sad': 2}, {'happy': 14, 'neutral': 7, 'fear': 6}, {'happy': 2}]
1/1 [==============================] - 0s 28ms/step
emotion_prediction = happy
emotion_predictions: [{'happy': 14, 'fear': 7, 'neutral': 2, 'sad': 2}, {'happy': 14, 'neutral': 7, 'fear': 6}, {'happy': 3}]
1/1 

...

1/1 [==============================] - 0s 28ms/step
emotion_prediction = neutral
emotion_predictions: [{'happy': 14, 'fear': 7, 'neutral': 2, 'sad': 2}, {'happy': 14, 'neutral': 7, 'fear': 6}, {'happy': 14, 'angry': 3, 'surprise': 2, 'sad': 1, 'fear': 6}, {'happy': 9, 'surprise': 10, 'fear': 5, 'neutral': 1}, {'surprise': 17}, {'surprise': 11, 'sad': 2, 'fear': 4, 'disgust': 1, 'happy': 1, 'angry': 1, 'neutral': 1}, {'surprise': 3, 'angry': 7, 'happy': 3, 'sad': 3, 'fear': 7, 'neutral': 5}, {'surprise': 3, 'sad': 3, 'neutral': 6, 'fear': 9, 'happy': 3, 'angry': 1}, {'surprise': 8, 'neutral': 7, 'angry': 2, 'fear': 8, 'sad': 1, 'happy': 1}, {'surprise': 1, 'fear': 8, 'neutral': 4, 'sad': 3, 'happy': 7, 'angry': 1}, {'fear': 7, 'sad': 2, 'angry': 2, 'neutral': 4, 'happy': 7}, {'surprise': 5, 'fear': 12, 'angry': 3, 'sad': 1, 'neutral': 1, 'disgust': 1, 'happy': 1}, {'happy': 4, 'fear': 1, 'angry': 3, 'neutral': 9, 'sad': 2}, {'neutral': 8, 'surprise': 8, 'fear': 3, 'sad': 3}, {'angry': 2

1/1 [==============================] - 0s 27ms/step
emotion_prediction = neutral
emotion_predictions: [{'happy': 14, 'fear': 7, 'neutral': 2, 'sad': 2}, {'happy': 14, 'neutral': 7, 'fear': 6}, {'happy': 14, 'angry': 3, 'surprise': 2, 'sad': 1, 'fear': 6}, {'happy': 9, 'surprise': 10, 'fear': 5, 'neutral': 1}, {'surprise': 17}, {'surprise': 11, 'sad': 2, 'fear': 4, 'disgust': 1, 'happy': 1, 'angry': 1, 'neutral': 1}, {'surprise': 3, 'angry': 7, 'happy': 3, 'sad': 3, 'fear': 7, 'neutral': 5}, {'surprise': 3, 'sad': 3, 'neutral': 6, 'fear': 9, 'happy': 3, 'angry': 1}, {'surprise': 8, 'neutral': 7, 'angry': 2, 'fear': 8, 'sad': 1, 'happy': 1}, {'surprise': 1, 'fear': 8, 'neutral': 4, 'sad': 3, 'happy': 7, 'angry': 1}, {'fear': 7, 'sad': 2, 'angry': 2, 'neutral': 4, 'happy': 7}, {'surprise': 5, 'fear': 12, 'angry': 3, 'sad': 1, 'neutral': 1, 'disgust': 1, 'happy': 1}, {'happy': 4, 'fear': 1, 'angry': 3, 'neutral': 9, 'sad': 2}, {'neutral': 8, 'surprise': 8, 'fear': 3, 'sad': 3}, {'angry': 2

1/1 [==============================] - 0s 28ms/step
emotion_prediction = neutral
emotion_predictions: [{'happy': 14, 'fear': 7, 'neutral': 2, 'sad': 2}, {'happy': 14, 'neutral': 7, 'fear': 6}, {'happy': 14, 'angry': 3, 'surprise': 2, 'sad': 1, 'fear': 6}, {'happy': 9, 'surprise': 10, 'fear': 5, 'neutral': 1}, {'surprise': 17}, {'surprise': 11, 'sad': 2, 'fear': 4, 'disgust': 1, 'happy': 1, 'angry': 1, 'neutral': 1}, {'surprise': 3, 'angry': 7, 'happy': 3, 'sad': 3, 'fear': 7, 'neutral': 5}, {'surprise': 3, 'sad': 3, 'neutral': 6, 'fear': 9, 'happy': 3, 'angry': 1}, {'surprise': 8, 'neutral': 7, 'angry': 2, 'fear': 8, 'sad': 1, 'happy': 1}, {'surprise': 1, 'fear': 8, 'neutral': 4, 'sad': 3, 'happy': 7, 'angry': 1}, {'fear': 7, 'sad': 2, 'angry': 2, 'neutral': 4, 'happy': 7}, {'surprise': 5, 'fear': 12, 'angry': 3, 'sad': 1, 'neutral': 1, 'disgust': 1, 'happy': 1}, {'happy': 4, 'fear': 1, 'angry': 3, 'neutral': 9, 'sad': 2}, {'neutral': 8, 'surprise': 8, 'fear': 3, 'sad': 3}, {'angry': 2

127.0.0.1 - - [28/Jun/2023 01:42:49] "GET /stop_model HTTP/1.1" 302 -
127.0.0.1 - - [28/Jun/2023 01:42:49] "GET /predictions HTTP/1.1" 200 -


total: 25
image_predictions: {'happy': 14, 'fear': 7, 'neutral': 2, 'sad': 2}
updated_dict: {'angry': 0.0, 'disgust': 0.0, 'fear': 28.0, 'happy': 56.0, 'sad': 8.0, 'surprise': 0.0, 'neutral': 8.0}
total: 27
image_predictions: {'happy': 14, 'neutral': 7, 'fear': 6}
updated_dict: {'angry': 0.0, 'disgust': 0.0, 'fear': 22.22, 'happy': 51.85, 'sad': 0.0, 'surprise': 0.0, 'neutral': 25.93}
total: 26
image_predictions: {'happy': 14, 'angry': 3, 'surprise': 2, 'sad': 1, 'fear': 6}
updated_dict: {'angry': 11.54, 'disgust': 0.0, 'fear': 23.08, 'happy': 53.85, 'sad': 3.85, 'surprise': 7.69, 'neutral': 0.0}
total: 25
image_predictions: {'happy': 9, 'surprise': 10, 'fear': 5, 'neutral': 1}
updated_dict: {'angry': 0.0, 'disgust': 0.0, 'fear': 20.0, 'happy': 36.0, 'sad': 0.0, 'surprise': 40.0, 'neutral': 4.0}
total: 17
image_predictions: {'surprise': 17}
updated_dict: {'angry': 0.0, 'disgust': 0.0, 'fear': 0.0, 'happy': 0.0, 'sad': 0.0, 'surprise': 100.0, 'neutral': 0.0}
total: 21
image_predictions: